In [1]:
from gezi.common import *
from gensim.models import Word2Vec
sys.path.append('..')
gezi.set_pandas()
# gezi.set_pandas_widder()
from src.config import *
gezi.init_flags()

In [2]:
root = FLAGS.root

In [3]:
def create_df(folder, workers=80):
  def _create_df(fpath):
    df = pd.read_json(fpath, dtype={'cell_type': 'category', 'source': 'str'}).reset_index().rename({"index":"cell_id"}, axis=1)
    df["id"] = fpath.rsplit(".", 1)[0].rsplit("/", 1)[-1]
    return df
  dfs = gezi.prun(_create_df, glob.glob(f'{folder}/*.json'), workers)
  df = pd.concat(dfs)
  df['source'] = df.source.apply(lambda x: x.replace('\n', BR))
  return df

In [4]:
workers = 80
train_file = f'{FLAGS.root}/train2.fea'
if os.path.exists(train_file):
  df = pd.read_feather(train_file)
else:
  df = create_df(f'{FLAGS.root}/train', workers)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
if not 'input_ids' in df.columns:
  if tokenizer.convert_tokens_to_ids(BR) == tokenizer.unk_token_id:
    assert len(BR) == 1
    tokenizer.add_tokens([BR], special_tokens=False)
  input_ids_list = gezi.prun(lambda x: tokenizer(x).input_ids, df.source.values, 80, desc='tokenize')
  df['input_ids'] = input_ids_list
  df['tokens'] = gezi.prun(tokenizer.convert_ids_to_tokens, df.input_ids.values, 80, desc='convert_ids_to_tokens')
  df.reset_index().to_feather(train_file)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
df.head()

,index,cell_id,cell_type,source,id,input_ids,tokens
0,0,1c7a5a71,code,"import warningsʶwarnings.filterwarnings(""ignore"")ʶʶimport sqlite3ʶimport pandas as pdʶimport numpy as npʶimport nltkʶimport stringʶimport matplotlib.pyplot as pltʶimport seaborn as snsʶfrom sklearn.feature_extraction.text import TfidfTransformerʶfrom sklearn.feature_extraction.text import TfidfVectorizerʶʶfrom sklearn.feature_extraction.text import CountVectorizerʶfrom sklearn.metrics import confusion_matrixʶfrom sklearn import metricsʶfrom sklearn.metrics import roc_curve, aucʶfrom nltk.ste...",a3baf6134c8506,"[1, 6306, 11917, 128001, 11917, 260, 28865, 60656, 268, 555, 309, 66478, 309, 285, 128001, 128001, 6306, 120474, 508, 128001, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 90668, 39501, 128001, 6306, 4022, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 292, 33566, 29274, 260, 51532, 616, 113492, 260, 12948, 6306, 897, 59426, 1892, 86911, 649, 128001, 292, 33566, 29274, 260, 51532, 61...","[[CLS], ▁import, ▁warnings, ʶ, ▁warnings, ., filter, warning, s, (, "", ignore, "", ), ʶ, ʶ, ▁import, ▁sqlite, 3, ʶ, ▁import, ▁pandas, ▁as, ▁p, d, ʶ, ▁import, ▁num, py, ▁as, ▁np, ʶ, ▁import, ▁nl, tk, ʶ, ▁import, ▁string, ʶ, ▁import, ▁mat, plot, lib, ., py, plot, ▁as, ▁pl, t, ʶ, ▁import, ▁sea, born, ▁as, ▁sn, s, ʶ, ▁from, ▁sk, learn, ., feature, _, extraction, ., text, ▁import, ▁T, fid, f, Transform, er, ʶ, ▁from, ▁sk, learn, ., feature, _, extraction, ., text, ▁import, ▁T, fid, f, Vector, izer..."
1,1,68f71f96,code,"# using the SQLite Table to read data.ʶcon = sqlite3.connect('../input/database.sqlite')ʶ#con = sqlite3.connect('database.sqlite') ʶʶ#filtering only positive and negative reviews i.e. ʶ# not taking into consideration those reviews with Score=3ʶfiltered_data = pd.read_sql_query(""""""SELECT * FROM Reviews WHERE Score != 3 LIMIT 100000"""""", con) ʶʶ# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.ʶdef partition(x):ʶ if x < 3:ʶ return 0ʶ return...",a3baf6134c8506,"[1, 953, 478, 262, 78550, 3751, 264, 623, 514, 260, 128001, 4636, 1842, 120474, 508, 260, 30407, 555, 280, 260, 260, 320, 42177, 320, 54698, 260, 51972, 5936, 280, 285, 128001, 953, 5699, 1842, 120474, 508, 260, 30407, 555, 280, 54698, 260, 51972, 5936, 280, 285, 128001, 128001, 953, 28865, 510, 364, 1453, 263, 2330, 1937, 584, 260, 473, 260, 128001, 953, 298, 787, 352, 3937, 421, 1937, 275, 13938, 1510, 508, 128001, 16334, 616, 9832, 1842, 845, 407, 260, 8523, 616, 51972, 616, 47975, 555, 3...","[[CLS], ▁#, ▁using, ▁the, ▁SQLite, ▁Table, ▁to, ▁read, ▁data, ., ʶ, ▁con, ▁=, ▁sqlite, 3, ., connect, (, ', ., ., /, input, /, database, ., sql, ite, ', ), ʶ, ▁#, con, ▁=, ▁sqlite, 3, ., connect, (, ', database, ., sql, ite, ', ), ʶ, ʶ, ▁#, filter, ing, ▁only, ▁positive, ▁and, ▁negative, ▁reviews, ▁i, ., e, ., ʶ, ▁#, ▁not, ▁taking, ▁into, ▁consideration, ▁those, ▁reviews, ▁with, ▁Score, =, 3, ʶ, ▁filtered, _, data, ▁=, ▁p, d, ., read, _, sql, _, query, (, "", "", "", SELECT, ▁*, ▁FROM, ▁Reviews..."
2,2,f0a7f5c4,code,"display = pd.read_sql_query("""""" SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*) FROM Reviews GROUP BY UserId HAVING COUNT(*)>1 """""", con)ʶprint(display.shape)ʶdisplay.head()ʶ",a3baf6134c8506,"[1, 1689, 1842, 845, 407, 260, 8523, 616, 51972, 616, 47975, 555, 309, 309, 309, 45786, 5675, 29935, 261, 4899, 29935, 261, 12028, 15303, 261, 2210, 261, 13938, 261, 7655, 261, 73857, 555, 1225, 285, 11371, 8939, 30686, 9506, 5675, 29935, 92773, 73857, 555, 1225, 285, 1504, 435, 307, 309, 309, 261, 4636, 285, 128001, 2118, 555, 35459, 260, 29753, 285, 128001, 1689, 260, 5563, 555, 285, 128001, 2]","[[CLS], ▁display, ▁=, ▁p, d, ., read, _, sql, _, query, (, "", "", "", ▁SELECT, ▁User, Id, ,, ▁Product, Id, ,, ▁Profile, Name, ,, ▁Time, ,, ▁Score, ,, ▁Text, ,, ▁COUNT, (, *, ), ▁FROM, ▁Reviews, ▁GROUP, ▁BY, ▁User, Id, ▁HAVING, ▁COUN

In [7]:
# 10w using 3.14min 1 epoch for emb 256
# for emb 128 only 0.3min
def gen_w2v(df, tokenizer, name='tokens', window=16, min_count=5, emb_dim=256, limit=0):
  sentences = [list(x) for x in df[name].values]
  ic(len(sentences))

  if limit:
    sentences = sentences[:limit]
    ic(len(sentences))
    name = name + f'.limit{limit}'
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=cpu_count(), epochs=10, callbacks=[monitor])
  
  root = f'{FLAGS.root}/w2v/{emb_dim}'
  ofile = f'{root}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  gezi.save(w2v, ofile)
  emb = np.random.uniform(-0.05, 0.05,(tokenizer.vocab_size, emb_dim))
  for i in range(tokenizer.vocab_size):
    token = tokenizer.convert_ids_to_tokens(i)
    if token in w2v.wv:
      emb[i] = w2v.wv[token]
  ofile = f'{root}/{name}.npy'
  np.save(ofile, emb)
  return w2v

In [9]:
gen_w2v(df, tokenizer, emb_dim=128, limit=100000)

[06/23/22 20:54:06] 1419350845.py:5 in gen_w2v()
                    len(sentences): 6370646
[06/23/22 20:54:08] 1419350845.py:9 in gen_w2v()
                    len(sentences): 100000


name: tokens.limit100000 epoch: 1 model loss: 0.0 elapsed minutes: 2.47
name: tokens.limit100000 epoch: 2 model loss: 0.0 elapsed minutes: 2.49
name: tokens.limit100000 epoch: 3 model loss: 0.0 elapsed minutes: 2.43
name: tokens.limit100000 epoch: 4 model loss: 0.0 elapsed minutes: 2.21
name: tokens.limit100000 epoch: 5 model loss: 0.0 elapsed minutes: 2.25
name: tokens.limit100000 epoch: 6 model loss: 0.0 elapsed minutes: 2.49
name: tokens.limit100000 epoch: 7 model loss: 0.0 elapsed minutes: 2.33
name: tokens.limit100000 epoch: 8 model loss: 0.0 elapsed minutes: 2.30
name: tokens.limit100000 epoch: 9 model loss: 0.0 elapsed minutes: 2.33
name: tokens.limit100000 epoch: 10 model loss: 0.0 elapsed minutes: 2.22


In [10]:
gen_w2v(df, tokenizer, emb_dim=128, limit=1000000)

[06/23/22 21:18:20] 1419350845.py:5 in gen_w2v()
                    len(sentences): 6370646
[06/23/22 21:18:22] 1419350845.py:9 in gen_w2v()
                    len(sentences): 1000000


name: tokens.limit1000000 epoch: 1 model loss: 0.0 elapsed minutes: 3.62
name: tokens.limit1000000 epoch: 2 model loss: 0.0 elapsed minutes: 3.42
name: tokens.limit1000000 epoch: 3 model loss: 0.0 elapsed minutes: 3.40
name: tokens.limit1000000 epoch: 4 model loss: 0.0 elapsed minutes: 3.39
name: tokens.limit1000000 epoch: 5 model loss: 0.0 elapsed minutes: 3.38
name: tokens.limit1000000 epoch: 6 model loss: 0.0 elapsed minutes: 3.36
name: tokens.limit1000000 epoch: 7 model loss: 0.0 elapsed minutes: 3.36
name: tokens.limit1000000 epoch: 8 model loss: 0.0 elapsed minutes: 3.37
name: tokens.limit1000000 epoch: 9 model loss: 0.0 elapsed minutes: 3.21
name: tokens.limit1000000 epoch: 10 model loss: 0.0 elapsed minutes: 3.02


In [11]:
df['n_tokens'] = df.tokens.apply(len)

In [12]:
df.describe(PERCENTILES2)

,index,n_tokens
count,6370646.0000,6370646.0000
mean,39.9360,108.0791
std,49.0379,1746.4885
min,0.0000,3.0000
25%,11.0000,16.0000
50%,26.0000,40.0000
75%,51.0000,97.0000
90%,89.0000,213.0000
95%,124.0000,326.0000
99%,231.0000,813.0000


In [13]:
df.head(1000000).describe(PERCENTILES2)

,index,n_tokens
count,1000000.0000,1000000.0000
mean,40.2587,106.8540
std,48.9060,1616.4606
min,0.0000,3.0000
25%,11.0000,16.0000
50%,26.0000,40.0000
75%,52.0000,97.0000
90%,90.0000,212.0000
95%,125.0000,325.0000
99%,232.0000,816.0000


In [14]:
gen_w2v(df, tokenizer, emb_dim=128, limit=3000000)

[06/23/22 22:18:16] 1419350845.py:5 in gen_w2v()
                    len(sentences): 6370646
[06/23/22 22:18:17] 1419350845.py:9 in gen_w2v()
                    len(sentences): 3000000


name: tokens.limit3000000 epoch: 1 model loss: 0.0 elapsed minutes: 61.66
name: tokens.limit3000000 epoch: 2 model loss: 0.0 elapsed minutes: 63.36
name: tokens.limit3000000 epoch: 3 model loss: 0.0 elapsed minutes: 70.73
name: tokens.limit3000000 epoch: 4 model loss: 0.0 elapsed minutes: 70.91
name: tokens.limit3000000 epoch: 5 model loss: 0.0 elapsed minutes: 69.81
name: tokens.limit3000000 epoch: 6 model loss: 0.0 elapsed minutes: 69.59
name: tokens.limit3000000 epoch: 7 model loss: 0.0 elapsed minutes: 69.15
name: tokens.limit3000000 epoch: 8 model loss: 0.0 elapsed minutes: 68.21
name: tokens.limit3000000 epoch: 9 model loss: 0.0 elapsed minutes: 67.85
name: tokens.limit3000000 epoch: 10 model loss: 0.0 elapsed minutes: 69.11
